In [2]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import scipy.stats as ss
from scipy.optimize import curve_fit
from ntuple_lib import *
beautify_plots(plt)
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

In [7]:
def t_stats_tracks_in_vtxs(vertex_tracks_idx, track_t, v_zs):
    all_ts = []
    avg_ts = []
    rms_ts = []
    stat_zs = []
    
    for i, (event, event_zs) in enumerate(zip(vertex_tracks_idx, v_zs)):
        event_all_t = []
        event_avg_t = []
        event_rms_t = []
        event_stat_zs = []
        
        for vertex_tracks, z in zip(event, event_zs):
            if len(vertex_tracks) > 0:
                ts = np.array(track_t[i][vertex_tracks])
                ts = ts[np.logical_and(ts != -1e3, ts != -1e6)]
                event_all_t.append(ts)
                avg_t = np.mean(ts)
                rms_t = np.sqrt(np.mean(ts**2))
                event_avg_t.append(avg_t)
                event_rms_t.append(rms_t)
                event_stat_zs.append(z)
        all_ts.append(np.array(event_all_t))
        avg_ts.append(np.array(event_avg_t))
        rms_ts.append(np.array(event_rms_t))
        stat_zs.append(np.array(event_stat_zs))
    return np.array(all_ts, dtype='object'), np.array(avg_ts, dtype='object'), np.array(rms_ts, dtype='object'), np.array(stat_zs, dtype='object')

def t_stats_tracks_in_jets(vertex_tracks_idx, track_t):
    avg_ts = []
    rms_ts = []
    
    for i, event in enumerate(vertex_tracks_idx):
        event_avg_t = []
        event_rms_t = []
        for vertex_tracks in event:
            if len(vertex_tracks) > 0:
                ts = np.array(track_t[i][vertex_tracks])
                ts = ts[np.logical_and(ts != -1e3, ts != -1e6)]
                avg_t = np.mean(ts)
                rms_t = np.sqrt(np.mean(ts**2))
                event_avg_t.append(avg_t)
                event_rms_t.append(rms_t)
        avg_ts.append(event_avg_t)
        rms_ts.append(event_rms_t)
    return np.array(avg_ts, dtype='object'), np.array(rms_ts, dtype='object')
    
def t_stats_tracks_in_vtxs_pTcut(track_t, vertex_tracks_idx, vertex_zs, vertex_sumPt2, sumPt2_min):
    all_ts = []
    avg_ts = []
    rms_ts = []
    stat_zs = []
    
    for i, (event_vertex_track_idx, event_vertex_zs, event_vertex_sumPt2) in enumerate(zip(vertex_tracks_idx, vertex_zs, vertex_sumPt2)):
        event_all_ts =  []
        event_avg_t =   []
        event_rms_t =   []
        event_stat_zs = []
        
        hard_enough_vertex_idx = event_vertex_sumPt2 > sumPt2_min
        
        for vertex_tracks, z in zip(event_vertex_track_idx[hard_enough_vertex_idx], event_vertex_zs[hard_enough_vertex_idx]):
            if len(vertex_tracks) > 0:
                ts = np.array(track_t[i][vertex_tracks])
                ts = ts[np.logical_and(ts != -1e3, ts != -1e6)]
                event_all_ts.append(ts)
                
                ##Ugly hack
                event_all_ts.append(np.array([])) 
                
                avg_t = np.mean(ts)
                rms_t = np.sqrt(np.mean(ts**2))
                event_avg_t.append(avg_t)
                event_rms_t.append(rms_t)
                event_stat_zs.append(z)
        avg_ts.append (np.array(event_avg_t))
        rms_ts.append (np.array(event_rms_t))
        stat_zs.append(np.array(event_stat_zs))
        all_ts.append (np.array(event_all_ts, dtype='object'))
    return np.array(all_ts, dtype='object'), np.array(avg_ts, dtype='object'), np.array(rms_ts, dtype='object'), np.array(stat_zs, dtype='object')

In [8]:
files = ['ntuples_condor/OutDir_{}/hist-Rel21sample.root'.format(i) for i in range(1, 2)]
track_t = np.array([], dtype='object')
track_t30 = np.array([], dtype='object')
track_t90 = np.array([], dtype='object')
track_t180 = np.array([], dtype='object')

reco_vertex_track_idxs = np.array([], dtype='object')
truth_vertex_track_idxs = np.array([], dtype='object')
jet_track_idxs = np.array([], dtype='object')

reco_vertex_sumPt2 = np.array([], dtype='object')

reco_vertex_z = np.array([], dtype='object')
truth_vertex_z = np.array([], dtype='object')

for file in files:
    root_file = uproot.open(file)
    for key in root_file.keys():
        if key.startswith('EventTree'):
            tree = root_file[key]
            track_t = np.concatenate((track_t, tree['track_t'].array(library='np')))
            track_t30 = np.concatenate((track_t30, tree['track_t30'].array(library='np')))
            track_t90 = np.concatenate((track_t90, tree['track_t90'].array(library='np')))
            track_t180 = np.concatenate((track_t180, tree['track_t180'].array(library='np')))
            
            reco_vertex_track_idxs = np.concatenate((reco_vertex_track_idxs, tree['recovertex_tracks_idx'].array(library='np')))
            truth_vertex_track_idxs = np.concatenate((truth_vertex_track_idxs, tree['truthvertex_tracks_idx'].array(library='np')))
            jet_track_idxs = np.concatenate((jet_track_idxs, tree['jet_tracks_idx'].array(library='np')))
            
            reco_vertex_sumPt2 = np.concatenate((reco_vertex_sumPt2, tree['recovertex_sumPt2'].array(library='np')))
            
            reco_vertex_z = np.concatenate((reco_vertex_z, tree['recovertex_z'].array(library='np')))
            truth_vertex_z = np.concatenate((truth_vertex_z, tree['truthvertex_z'].array(library='np')))

In [9]:
rev_all_ts, rev_avg_ts, rev_rms_ts, rev_stat_z = t_stats_tracks_in_vtxs(reco_vertex_track_idxs, track_t, reco_vertex_z)
rev_all_ts30, rev_avg_ts30, rev_rms_ts30, rev_stat_z30 = t_stats_tracks_in_vtxs(reco_vertex_track_idxs, track_t30, reco_vertex_z)
rev_all_ts90, rev_avg_ts90, rev_rms_ts90, rev_stat_z90 = t_stats_tracks_in_vtxs(reco_vertex_track_idxs, track_t90, reco_vertex_z)
rev_all_ts180, rev_avg_ts180, rev_rms_ts180, rev_stat_z180 = t_stats_tracks_in_vtxs(reco_vertex_track_idxs, track_t180, reco_vertex_z)

trv_all_ts, trv_avg_ts, trv_rms_ts, trv_stat_z = t_stats_tracks_in_vtxs(truth_vertex_track_idxs, track_t, truth_vertex_z)

jet_avg_ts, jet_rms_ts = t_stats_tracks_in_jets(jet_track_idxs, track_t)

trv_all_ts_flat = flatten_array(trv_all_ts)
rev_all_ts_flat = flatten_array(rev_all_ts)
rev_all_ts30_flat = flatten_array(rev_all_ts30)
rev_all_ts90_flat = flatten_array(rev_all_ts90)
rev_all_ts180_flat = flatten_array(rev_all_ts180)

trv_all_ts_flat2 = flatten_array(trv_all_ts_flat)
rev_all_ts_flat2 = flatten_array(rev_all_ts_flat)
rev_all_ts30_flat2 = flatten_array(rev_all_ts30_flat)
rev_all_ts90_flat2 = flatten_array(rev_all_ts90_flat)
rev_all_ts180_flat2 = flatten_array(rev_all_ts180_flat)

rev_avg_ts_flat = flatten_array(rev_avg_ts)
trv_avg_ts_flat = flatten_array(trv_avg_ts)
jet_avg_ts_flat = flatten_array(jet_avg_ts)

rev_avg_ts30_flat = flatten_array(rev_avg_ts30)
rev_avg_ts90_flat = flatten_array(rev_avg_ts90)
rev_avg_ts180_flat = flatten_array(rev_avg_ts180)

rev_rms_ts_flat = flatten_array(rev_rms_ts)
trv_rms_ts_flat = flatten_array(trv_rms_ts)
jet_rms_ts_flat = flatten_array(jet_rms_ts)

rev_stat_z_flat = flatten_array(rev_stat_z)
trv_stat_z_flat = flatten_array(trv_stat_z)

<ipython-input-7-6a6e98eee098>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_ts.append(np.array(event_all_t))
/home/ttlindkvist/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ttlindkvist/.local/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


In [79]:
def find_t0(ts, sigma_cutoff = 3):
    if len(ts) == 0:
        return [], []
    
    ts_left = np.array([])
    
    tavg = np.mean(ts)
    tstd = np.std(ts)
    
    outside = ts[np.abs(ts-tavg) > sigma_cutoff*tstd]
    ts_left = np.concatenate((ts_left, outside))
    while len(outside) > 0:
        ts = ts[np.abs(ts-tavg) <= sigma_cutoff*tstd]
        tavg = np.mean(ts)
        tstd = np.std(ts)
            
        outside = ts[np.abs(ts-tavg) > sigma_cutoff*tstd]
        ts_left = np.concatenate((ts_left, outside))
    
    next_layer_tavg, next_layer_nTracks = find_t0(ts_left, sigma_cutoff)
    return np.concatenate(([tavg], next_layer_tavg)), np.concatenate(([len(ts)], next_layer_nTracks))

In [80]:
n = 1
np.sort(rev_all_ts_flat[n]), np.mean(rev_all_ts_flat[n])

(array([-0.21531864, -0.21531864, -0.21531864, -0.21531864, -0.21531864,
        -0.21531864, -0.21531864, -0.21531864, -0.21531864, -0.21531864,
        -0.21531864, -0.21531864, -0.21531864, -0.21531864, -0.21531864,
        -0.1101291 , -0.1101291 , -0.1101291 , -0.1101291 , -0.1101291 ,
        -0.1101291 , -0.1101291 , -0.06458008, -0.06458008,  0.04610058,
         0.04610058,  0.04610058,  0.04610058,  0.04610058,  0.04610058,
         0.04610058,  0.04610058,  0.04610058,  0.04610058,  0.04610058,
         0.04610058,  0.18919699], dtype=float32),
 -0.09155242)

In [84]:
find_t0(rev_all_ts_flat[n], 1)

(array([-0.06458008, -0.21531864,  0.04610058, -0.21087769]),
 array([4., 6., 5., 1.]))

In [82]:
n = 10
np.sort(rev_all_ts90_flat[n]), np.mean(rev_all_ts90_flat[n])

(array([-0.3418961 , -0.32307532, -0.27557522, -0.2330861 , -0.17797486,
        -0.1628631 , -0.1466363 , -0.14475521, -0.0747785 , -0.07257065,
        -0.05632129, -0.03016583,  0.01342583,  0.02077092,  0.10800549,
         0.11426248], dtype=float32),
 -0.11145212)

In [83]:
find_t0(rev_all_ts90_flat[n], 1.6)

(array([-0.08318691, -0.15856079]), array([10.,  6.]))